In [72]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [13]:
data = pd.read_csv('../data/retail_store_inventory.csv')
data.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer
3,2022-01-01,S001,P0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn
4,2022-01-01,S001,P0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer


# Common preprocessing

## Changing the data type of the Date column from object to datetime

In [ ]:
def date_to_datetime(df):
    """
    Convert a specified column in a DataFrame to datetime format.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    
    Returns:
    pd.DataFrame: The DataFrame with the specified column converted to datetime.
    """
    df = df.copy() # Avoid modifying the original DataFrame
    df['Date'] = pd.to_datetime(df['Date'])
    return df

In [15]:
# We will test the preprocessing function
preprocessed_data = date_to_datetime(data)
preprocessed_data.dtypes

Date                  datetime64[ns]
Store ID                      object
Product ID                    object
Category                      object
Region                        object
Inventory Level                int64
Units Sold                     int64
Units Ordered                  int64
Demand Forecast              float64
Price                        float64
Discount                       int64
Weather Condition             object
Holiday/Promotion              int64
Competitor Pricing           float64
Seasonality                   object
dtype: object

## Temporal features

In [17]:
def temp_features(df):
    """
    Extracts temporal features from a DataFrame with a 'Date' column.

    Parameters:
    df (pd.DataFrame): The DataFrame containing a 'Date' column of datetime type.

    Returns:
    pd.DataFrame: The DataFrame with additional temporal features.
    """
    df = df.copy()  # Avoid modifying the original DataFrame

    # Basic date features
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['Weekday'] = df['Date'].dt.weekday  # 0 = lunes

    # Additional features
    df['Week'] = df['Date'].dt.isocalendar().week
    df['DayOfYear'] = df['Date'].dt.dayofyear
    df['Quarter'] = df['Date'].dt.quarter
    df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
    df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)

    # cyclical features
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    df['Weekday_sin'] = np.sin(2 * np.pi * df['Weekday'] / 7)
    df['Weekday_cos'] = np.cos(2 * np.pi * df['Weekday'] / 7)

    return df

In [19]:
# Testing the temporal features function
preprocessed_data = temp_features(preprocessed_data)
preprocessed_data.columns

Index(['Date', 'Store ID', 'Product ID', 'Category', 'Region',
       'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast',
       'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion',
       'Competitor Pricing', 'Seasonality', 'Year', 'Month', 'Day', 'Weekday',
       'Week', 'DayOfYear', 'Quarter', 'IsMonthStart', 'IsMonthEnd',
       'Month_sin', 'Month_cos', 'Weekday_sin', 'Weekday_cos'],
      dtype='object')

## Encodign categorical values

In [ ]:
def encode_categorical(df):
    """
    Encode categorical columns in a DataFrame using label encoder.

    Parameters:
    df (pd.DataFrame): The DataFrame containing categorical columns.

    Returns:
    pd.DataFrame: The DataFrame with categorical columns encoded.
    """
    df = df.copy()
    cat_cols = df.select_dtypes(include='object').columns.tolist()
    
    encoders = {}
    encoded_cols = []

    for col in cat_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        encoders[col] = le
        encoded_cols.append(col)

    return df, encoders, encoded_cols

In [84]:
preprocessed_data = encode_categorical(data)[0] #We just need the encoded DataFrame
preprocessed_data.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,0,0,0,3,1,231,127,55,135.47,33.50,20,1,0,29.69,0
1,0,0,1,4,2,204,150,66,144.04,63.01,20,3,0,66.16,0
2,0,0,2,4,3,102,65,51,74.02,27.99,10,3,1,31.32,2
3,0,0,3,4,1,469,61,164,62.18,32.72,10,0,1,34.74,0
4,0,0,4,1,0,166,14,135,9.26,73.64,0,3,0,68.95,2


## Sorting the dataset by date and ID

In [73]:
def sort_by_date_and_id(df, date_col='Date', id_cols=['Store ID', 'Product ID']):
    """
    Sort a DataFrame by a specified date column and an ID column.

    Parameters:
    df (pd.DataFrame): The DataFrame to sort.
    date_column (str): The name of the date column to sort by.
    id_column (str): The name of the ID column to sort by.

    Returns:
    pd.DataFrame: The sorted DataFrame.
    """
    df = df.copy()  # Avoid modifying the original DataFrame
    return df.sort_values(by=id_cols + [date_col])

In [74]:
preprocessed_data = sort_by_date_and_id(data)
preprocessed_data.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn
100,2022-01-02,S001,P0001,Groceries,West,116,81,104,92.94,27.95,10,Cloudy,0,30.89,Spring
200,2022-01-03,S001,P0001,Electronics,West,154,5,189,5.36,62.70,20,Rainy,0,58.22,Winter
300,2022-01-04,S001,P0001,Groceries,South,85,58,193,52.87,77.88,15,Cloudy,1,75.99,Winter
400,2022-01-05,S001,P0001,Groceries,South,238,147,37,150.27,28.46,20,Sunny,1,29.40,Winter


## Creating lags 

In [ ]:
# Creating lags
def create_lags(df, lags=[1, 7, 14, 30], target_col = 'Inventory Level'):
    """
    Create lag features for a specified target column in a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    lags (list): A list of integers representing the lag periods to create.
    target_col (str): The name of the target column for which to create lags.

    Returns:
    pd.DataFrame: The DataFrame with lag features added.
    """
    df = df.copy()  # Avoid modifying the original DataFrame
    df = df.sort_values(by=['Store ID', 'Product ID', 'Date'])  # Ensure the DataFrame is sorted by Store ID, Product ID, and Date
    group_cols = ['Store ID', 'Product ID'] # we need to create lags for each store and product, each series will be shifted independently
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df.groupby(group_cols)[target_col].shift(lag)
    return df

In [47]:
# testing the lag function
preprocessed_data = create_lags(data)
preprocessed_data.head(8) # This will show the first 8 rows with the new lag features
# Note: The first few rows will have NaN values for the lag features since there are no previous values to reference, we can drop them later if needed.

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,Inventory Level_lag_1,Inventory Level_lag_7,Inventory Level_lag_14,Inventory Level_lag_30
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,NaN,NaN,NaN,NaN
100,2022-01-02,S001,P0001,Groceries,West,116,81,104,92.94,27.95,10,Cloudy,0,30.89,Spring,231.0,NaN,NaN,NaN
200,2022-01-03,S001,P0001,Electronics,West,154,5,189,5.36,62.70,20,Rainy,0,58.22,Winter,116.0,NaN,NaN,NaN
300,2022-01-04,S001,P0001,Groceries,South,85,58,193,52.87,77.88,15,Cloudy,1,75.99,Winter,154.0,NaN,NaN,NaN
400,2022-01-05,S001,P0001,Groceries,South,238,147,37,150.27,28.46,20,Sunny,1,29.40,Winter,85.0,NaN,NaN,NaN
500,2022-01-06,S001,P0001,Electronics,East,198,37,155,39.09,91.05,5,Snowy,0,87.30,Summer,238.0,NaN,NaN,NaN
600,2022-01-07,S001,P0001,Furniture,North,195,107,106,117.92,53.99,10,Sunny,1,58.80,Winter,198.0,NaN,NaN,NaN
700,2022-01-08,S001,P0001,Furniture,East,231,2,119,0.84,66.30,20,Cloudy,1,63.82,Winter,195.0,231.0,NaN,NaN


## Rolling means

In [ ]:
# We will add rolling means in order to smooth the trend
def add_rolling_means(df, window_sizes=[7, 14, 30], target_col='Inventory Level'):
    """
    Add rolling mean features to a DataFrame for specified window sizes.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    window_sizes (list): A list of integers representing the window sizes for rolling means.

    Returns:
    pd.DataFrame: The DataFrame with rolling mean features added.
    """
    df = df.copy()  # Avoid modifying the original DataFrame
    df = df.sort_values(by=['Store ID', 'Product ID', 'Date'])  # Ensure the DataFrame is sorted by Store ID, Product ID, and Date
    group_cols = ['Store ID', 'Product ID']  # we need to create rolling means for each store and product, each series will be smoothed independently
    for window in window_sizes:
        df[f'{target_col}_rolling_mean_{window}'] = df.groupby(group_cols)[target_col].transform(lambda x: x.rolling(window=window, min_periods=1).mean())
    return df

In [51]:
# testing the rolling means function

preprocessed_data = add_rolling_means(data)
preprocessed_data

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,Inventory Level_rolling_mean_7,Inventory Level_rolling_mean_14,Inventory Level_rolling_mean_30
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,231.000000,231.000000,231.000000
100,2022-01-02,S001,P0001,Groceries,West,116,81,104,92.94,27.95,10,Cloudy,0,30.89,Spring,173.500000,173.500000,173.500000
200,2022-01-03,S001,P0001,Electronics,West,154,5,189,5.36,62.70,20,Rainy,0,58.22,Winter,167.000000,167.000000,167.000000
300,2022-01-04,S001,P0001,Groceries,South,85,58,193,52.87,77.88,15,Cloudy,1,75.99,Winter,146.500000,146.500000,146.500000
400,2022-01-05,S001,P0001,Groceries,South,238,147,37,150.27,28.46,20,Sunny,1,29.40,Winter,164.800000,164.800000,164.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72699,2023-12-28,S005,P0020,Groceries,South,198,56,27,50.18,21.75,5,Sunny,1,25.29,Winter,317.714286,292.571429,290.466667
72799,2023-12-29,S005,P0020,Clothing,East,446,268,30,267.54,85.58,20,Sunny,1,87.63,Summer,328.428571,303.857143,294.266667
72899,2023-12-30,S005,P0020,Toys,North,251,149,181,162.92,79.48,10,Cloudy,1,82.69,Autumn,297.000000,316.000000,286.900000
72999,2023-12-31,S005,P0020,Furniture,East,64,40,99,59.69,90.79,5,Snowy,1,91.67,Winter,244.714286,295.714286,280.533333


## Cleaning the dataframe droping NaNs created by lag features

In [60]:
def clean_dataframe(df):
    """
    Clean a DataFrame by dropping rows with NaN values.

    Parameters:
    df (pd.DataFrame): The DataFrame to clean.

    Returns:
    pd.DataFrame: The cleaned DataFrame with NaN values dropped.
    """
    df = df.copy()  # Avoid modifying the original DataFrame
    return df.dropna()

In [54]:
# Testing the cleaning function by dropping NaN values created by lag features
preprocessed_data = create_lags(data)
preprocessed_data = clean_dataframe(preprocessed_data)
preprocessed_data.head()  # Display the first few rows of the cleaned DataFrame

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,Inventory Level_lag_1,Inventory Level_lag_7,Inventory Level_lag_14,Inventory Level_lag_30
3000,2022-01-31,S001,P0001,Clothing,West,410,200,152,212.24,70.78,0,Sunny,1,68.88,Summer,498.0,115.0,134.0,231.0
3100,2022-02-01,S001,P0001,Groceries,East,419,279,84,297.26,34.49,0,Sunny,0,33.85,Summer,410.0,348.0,339.0,116.0
3200,2022-02-02,S001,P0001,Electronics,South,415,38,149,53.13,52.49,5,Cloudy,0,48.25,Autumn,419.0,200.0,431.0,154.0
3300,2022-02-03,S001,P0001,Clothing,South,345,71,186,84.02,27.71,20,Rainy,1,28.96,Winter,415.0,410.0,154.0,85.0
3400,2022-02-04,S001,P0001,Furniture,South,121,25,25,37.72,16.84,15,Cloudy,0,12.53,Autumn,345.0,158.0,430.0,238.0


## Creating interaction features

In [ ]:
def interaction_features(df):
    """
    Create interaction features from numeric columns in a DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame. Must include the columns:
        'Price', 'Discount', 'Competitor Pricing', 'Inventory Level',
        'Units Ordered', and 'Units Sold'.

    Returns:
    pd.DataFrame: The DataFrame with new interaction features added:
        - price_discount
        - price_competitiveness
        - inventory_gap
    """
    df = df.copy()  # Avoid modifying the original DataFrame

    # Combinations
    df['price_discount'] = df['Price'] * df['Discount']
    df['price_competitiveness'] = df['Price'] / (df['Competitor Pricing'] + 1e-5)
    df['inventory_gap'] = df['Inventory Level'] - df['Units Ordered']
    return df

In [63]:
# testing interaction features
preprocessed_data = interaction_features(data)
preprocessed_data.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,price_discount,price_competitiveness,inventory_gap,inventory_turnover
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,670.0,1.128326,176,0.549784
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn,1260.2,0.952388,138,0.735294
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer,279.9,0.893678,51,0.637255
3,2022-01-01,S001,P0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn,327.2,0.941853,305,0.130064
4,2022-01-01,S001,P0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer,0.0,1.068020,31,0.084337


## Integrating all functiones in one step

In [85]:
# We will integrate all functions in one step in order to see where do we need to transform the skew data and avoid outliers
def common_preprocess_data(df):
    """
    Preprocess the DataFrame by applying a series of transformations:
    - Convert 'Date' to datetime format
    - Extract temporal features
    - Encode categorical variables
    - Sort by date and ID
    - Create lag features
    - Add rolling means
    - Clean the DataFrame by dropping NaN values
    - Create interaction features

    Parameters:
    df (pd.DataFrame): The input DataFrame.

    Returns:
    pd.DataFrame: The preprocessed DataFrame.
    """
    df = date_to_datetime(df)
    df = temp_features(df)
    df = encode_categorical(df)[0]
    df = sort_by_date_and_id(df)
    df = create_lags(df)
    df = add_rolling_means(df)
    df = clean_dataframe(df)
    df = interaction_features(df)
    
    return df

In [86]:
preprocessed_data = common_preprocess_data(data)
numerical_columns = preprocessed_data.select_dtypes(include='number')

for col in numerical_columns:
    skew = preprocessed_data[col].skew()
    if abs(skew) > 1:
        print(f"Column '{col}' is highly skewed with a skewness of {skew:.2f}.")
    elif abs(skew) > 0.5:
        print(f"Column '{col}' is moderately skewed with a skewness of {skew:.2f}.")

Column 'Units Sold' is moderately skewed with a skewness of 0.90.
Column 'Demand Forecast' is moderately skewed with a skewness of 0.89.
Column 'IsMonthStart' is highly skewed with a skewness of 5.12.
Column 'IsMonthEnd' is highly skewed with a skewness of 5.12.
Column 'price_discount' is moderately skewed with a skewness of 0.88.
Column 'price_competitiveness' is highly skewed with a skewness of 1.86.


If wee use a linear regression model we will need to transform the skewed variables